# AI-Generated Text Detection - Fine-tune BERT

In [2]:

# Run preprocessing script to ensure dataset is ready
!python3 ../scripts/data_preprocessing.py


Processed dataset saved to ../data/processed_hc3.csv with 85431 valid samples.


## Train BERT Model

In [12]:
%pip uninstall transformers -y
%pip install --upgrade transformers


Found existing installation: transformers 4.53.2
Uninstalling transformers-4.53.2:
  Successfully uninstalled transformers-4.53.2
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.53.2-py3-none-any.whl (10.8 MB)
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import transformers
print("Transformers version:", transformers.__version__)

/Users/ethanc/Library/Python/3.10/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Transformers version: 4.53.2


In [4]:
!which python

python not found


In [16]:
# diagnostics + run your train_bert script
import sys, inspect, subprocess

# 1) Which python executable are we using?
print("sys.executable:", sys.executable)

# 2) pip version & show transformers
print("pip version:", subprocess.check_output([sys.executable, "-m", "pip", "--version"], text=True).strip())
print(subprocess.check_output([sys.executable, "-m", "pip", "show", "transformers"], text=True))

# 3) transformers version & location
import transformers
print("transformers version:", transformers.__version__)
print("transformers loaded from:", transformers.__file__)

# 4) Inspect TrainingArguments signature
print("TrainingArguments signature:", inspect.signature(transformers.TrainingArguments))

# 5) Finally, invoke your script with the same interpreter
!{sys.executable} ../scripts/train_bert.py


sys.executable: /usr/local/bin/python3
pip version: pip 22.0.4 from /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip (python 3.10)
Name: transformers
Version: 4.53.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /Users/ethanc/Library/Python/3.10/lib/python/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 

transformers version: 4.53.2
transformers loaded from: /Users/ethanc/Library/Python/3.10/lib/python/site-packages/transformers/__init__.py
TrainingArguments signature: (output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool

In [20]:
# 1) Ensure accelerate is installed and up to date for Trainer compatibility
# %pip install --upgrade "accelerate>=0.26.0"

# 2) (Optional) Patch train_bert.py for eval_strategy if needed
import fileinput

for line in fileinput.input('../scripts/train_bert.py', inplace=True, backup='.bak'):
    print(line.replace('evaluation_strategy=', 'eval_strategy='), end='')

# 3) Re-invoke your script in the same environment
import sys
!{sys.executable} ../scripts/train_bert.py

Python(85056) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Transformers version: 4.53.2
Debug mode enabled: 200 training samples, 50 validation samples
tokenizer_config.json: 100%|█████████████████| 48.0/48.0 [00:00<00:00, 46.8kB/s]
vocab.txt: 100%|█████████████████████████████| 232k/232k [00:00<00:00, 2.43MB/s]
tokenizer.json: 100%|████████████████████████| 466k/466k [00:00<00:00, 10.0MB/s]
config.json: 100%|█████████████████████████████| 483/483 [00:00<00:00, 2.14MB/s]
Training on device: mps
model.safetensors: 100%|█████████████████████| 268M/268M [01:51<00:00, 2.41MB/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|                                                    | 0/25 [00:00<?, ?it/s]/Users/ethanc/Library/Python/3.10/lib/python/

## Load and Evaluate BERT Model

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

# Load the fine-tuned model and tokenizer
model_dir = '../models/bert_model'
tokenizer = DistilBertTokenizer.from_pretrained(model_dir)
model = DistilBertForSequenceClassification.from_pretrained(model_dir)

# Example evaluation on a sample text
sample_text = "This is an example sentence to check if the model thinks it is AI or human."
inputs = tokenizer(sample_text, return_tensors='pt', truncation=True, padding=True, max_length=128)
with torch.no_grad():
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
print(f"Sample text prediction: {'AI-generated' if prediction == 1 else 'Human-written'}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


ValueError: The state dictionary of the model you are trying to load is corrupted. Are you sure it was properly saved?